In [1]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [2]:
from zkstats.core import create_dummy, verifier_define_calculation, prover_gen_settings, setup, prover_gen_proof, verifier_verify, generate_data_commitment

In [3]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')
data_commitment_path = os.path.join('shared/data_commitment.json')
precal_witness_path = os.path.join('shared/precal_witness_arr.json')

=======================  ZK-STATS FLOW =======================

In [4]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')
create_dummy(data_path, dummy_data_path)


In [5]:
scales = [4]
# to conform to traditional regression, here only one column of x
selected_columns = ['x1', 'y']
generate_data_commitment(data_path, scales, data_commitment_path)

In [6]:
# Verifier/ data consumer side: send desired calculation
from zkstats.computation import computation_to_model, State


def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x, y = data[0], data[1]
    return s.linear_regression(x, y)

error = 0.05
_, prover_model = computation_to_model(computation, precal_witness_path, True, selected_columns, error)

prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model,prover_model_path, scales, "resources", settings_path)

/Users/jernkun/Desktop/zk-stats-lib/zkstats/computation.py:254: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  is_precise_aggregated = torch.tensor(1.0)
/Users/jernkun/Desktop/zk-stats-lib/zkstats/ops.py:542: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  x_one = torch.where((x_one[:,:,0] ==MagicNumber).unsqueeze(-1), torch.tensor([0.0]*x_one.size()[2]), x_one)
/Users/jernkun/Library/Caches/pypoetry/virtualenvs/zkstats-OJpceffF-py3.11/lib/python3.11/site-packages/torch/onnx/symbo

==== Generate & Calibrate Setting ====
scale:  [4]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":4,"param_scale":4,"scale_rebase_multiplier":1,"lookup_range":[-11594,226],"logrows":14,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Fixed","div_rebasing":false,"rebase_frac_zero_constants":false,"check_mode":"UNSAFE"},"num_rows":7872,"total_assignments":2189,"total_const_size":615,"model_instance_shapes":[[1],[1,2,1]],"model_output_scales":[0,4],"model_input_scales":[4,4],"module_sizes":{"kzg":[],"poseidon":[7872,[2]]},"required_lookups":[{"GreaterThan":{"a":0.0}},{"Cast":{"scale":16.0}}],"required_range_checks":[[-8,8]],"check_mode":"UNSAFE","version":"9.1.0","num_blinding_factors":null,"timestamp":1715324256685}


In [7]:
_, verifier_model = computation_to_model(computation, precal_witness_path, False, selected_columns, error)

verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path, verifier_model, verifier_model_path)

In [8]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

==== setting up ezkl ====
Time setup: 1.8887672424316406 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 1.5625
witness result 2 : 9.5625
==== Generating Proof ====
proof:  {'instances': [['78f113a919d1324cbee267b4320db42ee0170745a57013fa302f8139d7c36f18', '8dff683cfffbd97a4d94de7d897e03bb3f2fa18084beff4bbfbd152c2e2bcd16', '0100000000000000000000000000000000000000000000000000000000000000', '1900000000000000000000000000000000000000000000000000000000000000', '9900000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x10efc7ac36b39c51e5ca14db77d23efcad44a6cc25894e4e6f3241dbeba3109c0bd95c2e1efa4035c54afe0ec1e408ad91ca89fa35c682e85a120ea112b3e8780a8615b3a15a46c14f94ed14a2292140978ff43788da6efe43ea8f78711d688f171ecb383756fd177f72c3f4eb659c43b8981c142e9587935f6361ce6f41f67d23c0e44c78bd2e0dd0be93251b65ca7c241e97aa729cf2e812479282981e80f514e19ebf84c8ffb588bd1485b4db1f8b8d6deb635a63bd2a350b16e4c4ea8d111e171040368dd4dd9a4b9af5c0e4f1bb1d619d408a3

In [9]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, data_commitment_path)
print("Verifier gets result:", res)

Verifier gets result: [1.5625, 9.5625]
